In [5]:
import pandas as pd
import numpy as np
from prophet import Prophet
import hummingbird.ml

#### Create and fit the model

In [6]:
# Read the data and train a Prophet model
DATA = "../tests/resources/example_wp_log_peyton_manning.csv"
df = pd.read_csv(DATA)

model = Prophet()
model.fit(df)
        

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [7]:
# Convert into PyTorch using Hummingbird
 hb_model = hummingbird.ml.convert(model, "torch")

AttributeError: module 'hummingbird' has no attribute 'ml'

In [22]:
# Generate future dataset.
future = model.make_future_dataframe(periods=365)

In [23]:
# Trend prediction with prophet
prophet_trend = model.predict(future)["trend"]

In [24]:
# Trend prediction with Hummingbird
hb_trend = hb_model.predict(future)

In [25]:
# Check that the predictions match.
np.testing.assert_allclose(prophet_trend, hb_trend, rtol=1e-06, atol=1e-06)

In [26]:
%%timeit
prophet_trend = model.predict(future)

2.17 s ± 53.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%%timeit
hb_trend = hb_model.predict(future)

4.33 ms ± 434 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
# Generate onnx model. ONNX model requires inputs in numpy format.
future_np = (future.values - np.datetime64("1970-01-01T00:00:00.000000000")).astype(np.int64) / 1000000000
hb_onnx_model = hummingbird.ml.convert(model, "onnx", future_np)

/Users/mainterl/.pyenv/versions/3.6.7/lib/python3.6/site-packages/torch/tensor.py:590: RuntimeWarning:

Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).



In [29]:
%%timeit
hb_trend = hb_onnx_model.predict(future_np)

7.11 ms ± 23.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
